# Amazon SageMaker Workshop
### _**Data Preparation**_

---
In this part of the workshop we will prepare the data to later train our churn model.

---

## Contents

1. [Background](#Background) - Getting the rawata prepared in the previous lab.
2. [Prepare](#Prepare) - Prepare the data with [Amazon SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/)
    * [Creating features](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html)
    * [Creating analysis](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-analyses.html)
    * [Analyzing the data and features](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html)
3. [Submiting the data to Feature Store](#FeatureStore) - Store the features created in [Amazon SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/)
  
---

## Background

In the previous [Introduction lab](../0-Introduction/introduction.ipynb) we created a S3 bucket and uploaded the raw data to it.

Let's get started!

Get variables from previous configuration notebook:

In [ ]:
%store -r bucket
%store -r region
%store -r prefix
%store -r s3uri_raw
%store -r docker_image_name
%store -r framework_version
bucket, prefix, s3uri_raw, region, docker_image_name, framework_version

Let's import the libraries for this lab:

In [ ]:
import sagemaker

sess = sagemaker.Session()

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()
role

In [ ]:
import io
import os
import sys
import time
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display as dis
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from IPython import display 

# Importing Data on DataWrangler

To start, we will create a new flow and import the raw data to perform analysis and transformations on it. On the left menu, click on "Components and Registries", select "Data Wrangler" on the dropdown, and click "New Flow". This process is shown on the image below.

<img src="media/010-new_flow.png" width="30%" />

As soon as we click on "New Flow", Data Wrangler will be on a loading state. After a couple minutes you should be able to import the raw data on it. While we wait, we can rename our flow by right clicking on the flow tab and choosing "Rename Data Wrangler Flow...".

<img src="media/020-load_new_flow.png" width="100%" />

Let's call the file `churn.flow`

After Data Wrangler finishes loading, we may proceed importing our data. We'll be importing our data from Amazon S3. The following images guide us through the process.

<img src="media/030-importing_from_s3.png" width="100%" />

Using the search box or the table in the bottom, locate the S3 Bucket where our dataset is stored.

To remember the bucket and prefix run cell below:

In [ ]:
s3uri_raw

<img src="media/040-choosing_bucket.png" width="100%" />

Choose the "churn.csv" file, disable the "Enable sampling" checkbox, and click "Import". Feel free to check the data preview on the "Preview" session on the bottom!

<img src="media/050-choose_csv_preview.png" width="100%" />

# Data Analysis on Data Wrangler

Next, we are going to analyze our data by understanding feature distribution and how each of them impacts our target column. Data Wrangler allow us to perform such analysis inside user interface, let's start creating these analysis.

To start our analysis, we'll create a summary of our imported data. The summary can be rertieved by adding a new analysis on Data Wrangler. The following images show the step by step to create our table summary.

<img src="media/060-add_first_analysis.png" width="100%" />

<img src="media/070-table_summary.png" width="100%" />

As soon as you click on Preview or entering in your saved analysis, you'll see the table summary as shown in the image below.

<img src="media/080-table_summary_preview.png" width="100%" />

By modern standards, it’s a relatively small dataset, with only 5,000 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Continuing our data analysis, we can leverage different Analysis type on Data Wrangler. Next, we can use Histogram to understand our feature distribution and how it impacts our target value. Going back to the "Analyze" tab, we can add a new Analysis. Then, we can choose Histogram as the "Analysis type" and select a feature to explore its distribution. On the following images we've chosen `Account Length` as feature and colored it by our target variable `Churn?`.

<img src="media/090-analyze_tab_create_new_analysis.png" width="100%" />

<img src="media/100-account_length_histogram.png" width="100%" />

As we can see, we are able to check the distribution of our feature and how its distribution relates to our target value. Feel free to create new histograms for any other feature!

For our last analysis, we'll leverage the "Quick Model" analysis provided by Data Wrangler. This analysis trains a Random Forest algorithm on its own and calculates a feature importance score for each feature on our dataset. You can learn more about the "Quick Model" analysis on this [page](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-analyses.html#data-wrangler-quick-model) of the Amazon SageMaker Data Wrangler documentation.


<img src="media/110-quick_model_analysis.png" width="100%" />

The higher the score, the more important the feature is. Therefore, feature `Day Mins` is the most important feature on our dataset according to the "Quick Model" analysis.

# Data Transforms on Data Wrangler

Go back to the DAG view by clicking on `Prepare` tab (on the top  left).

Follow the instructions on the image below:  
1 - Click on the plus (+) button  
2 - Click Edit data types  

<img src="media/120-edit_data_types.png" width="100%" />

3 - Find the column you want to change  
4 - Select the desired type from the dropdown
5 - Click preview  
6 - Click Add

<img src="media/130-area_code_to_object.png" width="100%" />

Once you finish, click "Back to data flow" on the top right corner

### Now lets drop the Phone column by adding a Transform  
1 - Click the plus (+)  
2 - Add Transform

<img src="media/140-add_transform.png" width="100%" />

3 - Click on manage columns  
4 - Select the `Phone` column from the dropdown (as shown in step 2)  
5 - Click on preview  
6 - Click add

<img src="media/150-steps_drop_phone_col.png" width="100%" />

### Now lets Drop a few more columns  
I'll be dropping the first (Day Charge) as an example, just repeat the steps in the image below for the following columns:
* "Day Charge"
* "Eve Charge"
* "Night Charge"
* "Intl Charge"

<img src="media/160-adding_new_transform.png" width="100%" />

<img src="media/170-drop_column_pt2.png" width="100%" />

### Now lets do OneHot Encoding using a custom transform
You can copy the code for the custom transform here:
```python
import pandas as pd

model_data = pd.get_dummies(df)

df = pd.concat(
  [model_data["Churn?_True."],
   model_data.drop( ["Churn?_False.", "Churn?_True."], axis=1)],
  axis=1
).rename( 
  columns = {
    "Churn?_True.": "Churn"
  }
)
```

<img src="media/180-custom_transform.png" width="100%" />

# Exercise:

### Your turn!!

Create a few more features and run `Quick Model` analysis to see if the features are helping improve the performance (F1 score).

In [ ]:
# Create your own features and test them with Quick Model analysis






# Exporting Transformed data on Data Wrangler

After performing the transformations needed on our dataset, we'll export the transformed data to our S3 bucket. We are able to do so inside Data Wrangler UI by following the steps highlighted on the images below.

For this export step **just choose the  following transforms:**

<img src="media/190-select_transforms_to_export.png" width="100%" />

<img src="media/200-exporting_to_s3.png" width="100%" />

After selecting the `Save to S3` option, a new notebook will be displayed similar to the one presented on the image below.

<img src="media/210-export_to_s3_notebook.png" width="100%" />

We can proceed to `Run all cells`, as described in the notebook. The processing job will start and it will take a few minutes to finish. Upon completion, we'll see a similar output on the cell presented on the following image.

<img src="media/219-dw-notebook-run-all.png" width="100%" />

In the end of that notebook check that Processing Job is running (with the Data Wrangler Docker image):

<img src="media/220-processing_job_finished.png" width="100%" />

On the left menu, click on "Components and Registries", select "Experiments and trial" on the dropdown.

Select and double-click "Unassigned trial components":

<img src="media/225-open-experiments-processing.png" width="50%"/>

Select your processing job trial and open (right click and select "Open in trial details"):

<img src="media/225-find-processing-info.png" width="60%"/>

Go to the `Artifacts` tab, and **copy** the S3 URI of the output:

<img src="media/225-find-processing-output.png" width="100%" />

Paste your S3 URI below:

In [ ]:
s3uri_processed = "s3://YOUR-BUCKET/PATH/TO/OUTPUT/HERE.csv"

In [ ]:
s3uri_processed_file = sagemaker.s3.S3Downloader.list(s3uri_processed)[0]
s3uri_processed_file

If you want to check it in S3 console run the cell bellow and click the link:

In [ ]:
from IPython.core.display import display, HTML
from sagemaker.s3 import parse_s3_url

out_bucket, out_prefix = parse_s3_url(s3uri_processed_file)
out_path = os.path.dirname(out_prefix)
out_file = os.path.basename(out_prefix)

s3_url_placeholder = "https://s3.console.aws.amazon.com/s3/buckets/{}?&prefix={}/"
display(HTML(f"<a href={s3_url_placeholder.format(out_bucket, out_path)}>Go to S3 console and check output of Data Wrangler</a>"))

In the S3 console you should see:

<img src="media/230-download_transformed_data_s3.png" width="100%" />

(If you want to download the data to you computer follow the steps in the image above)

Let's download the data to Studio:

In [ ]:
sess.download_data(".", 
                   out_bucket, 
                   key_prefix=out_prefix)

Click the refresh button on Studio. You should see something like:

<img src="media/230-download_transformed_data_s3_local.png" width="50%" />

(The CSV file is downloaded)

In [ ]:
out_file

In [ ]:
model_data = pd.read_csv(out_file)
model_data.head()

Above we should see the transformed data with `Churn` in the first columns, the one-hot-encoded columns and so on.

Finally, let's break the data into **train, validation and test sets:**

In [ ]:
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)
train_data.shape, validation_data.shape, test_data.shape

In [ ]:
train_data.head(2)

In [ ]:
model_data.shape

Create CSV files for the 3 datasets:

In [ ]:
train_file_name = "train.csv"
validation_file_name = "validation.csv"
test_file_name = "test.csv"

train_data.to_csv(train_file_name , header=False, index=False)
validation_data.to_csv(validation_file_name, header=False, index=False)
test_data.to_csv(test_file_name, header=False, index=False)

Lastly, we'll upload these files to S3.

In [ ]:
# Return the URLs of the uploaded files, so they can be reviewed or used elsewhere
train_dir = f"{prefix}/data/train"
val_dir = f"{prefix}/data/validation"
test_dir = f"{prefix}/data/test"

s3uri_train = sagemaker.s3.S3Uploader.upload(train_file_name, f's3://{bucket}/{train_dir}')
s3uri_validation = sagemaker.s3.S3Uploader.upload(validation_file_name, f's3://{bucket}/{val_dir}')
s3uri_test = sagemaker.s3.S3Uploader.upload(test_file_name, f's3://{bucket}/{test_dir}')
s3uri_train, s3uri_validation, s3uri_test

Save the S3 URIs for the 3 datasets for later:

In [ ]:
%store s3uri_train
%store s3uri_validation
%store s3uri_test

# Wrap-up

Now that we finished the **data preparation lab**, let's make everything here re-usable. It may come in handy later (spoiler alert - when creating Pipelines)...

**Obs.:** We will use the `sagemaker.processing.Processor` and class, like you saw in the notebook generated by Data Wrangler to save the processed data to S3 and also the `sagemaker.sklearn.processing.SKLearnProcessor` for spliting the dataset.

In [ ]:
# Save Data Wrangler configurations for feature engineering
s3uri_code = f"s3://{bucket}/{prefix}/code"
s3uri_code

s3_dataprep_dw_code_uri = sagemaker.s3.S3Uploader.upload("churn.flow", s3uri_code)
%store s3_dataprep_dw_code_uri
s3_dataprep_dw_code_uri

Create Processing code for splitting data sets:

In [ ]:
%%writefile preprocess_split.py
"""Feature engineers the customer churn dataset."""
import argparse
import logging
import pathlib

import boto3
import numpy as np
import pandas as pd

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

def get_df(input_data_dir):
    logger.info("Reading downloaded data and putting it in a Pandas DF.")
    input_files = [ os.path.join(input_data_dir, file) for file in os.listdir(input_data_dir) ]
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the input ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(input_data_dir, "train"))
    raw_data = [ pd.read_csv(file, header=None, engine="python") for file in input_files ]
    
    return pd.concat(raw_data)

    
def get_files_df(input_s3_path) -> pd.DataFrame:
    bucket_name = input_data.split("/")[2]
    key = "/".join(input_data.split("/")[3:])
    
    base_dir = "/opt/ml/processing"
    input_data_dir = f"{base_dir}/data"
    pathlib.Path(input_data_dir).mkdir(parents=True, exist_ok=True)

    logger.info("Downloading data from bucket: %s, key: %s", bucket, key)
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(bucket_name) 
    
    for obj in bucket.objects.filter(Prefix = key):
        bucket.download_file(obj.key, f"{input_data_dir}/{obj.key}") # save to the data path

    # read in CSVs and return DF
    return get_df(input_data_dir)
    

if __name__ == "__main__":
    logger.info("Starting preprocessing.")
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-data", type=str, required=True)
    args = parser.parse_args()

    input_data = args.input_data
    
    # Download processed data and get pandas DF
    model_data = get_files_df(input_data) 
    
    # Split the data
    train_data, validation_data, test_data = np.split(
        model_data.sample(frac=1, random_state=1729),
        [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
    )

    pd.DataFrame(train_data).to_csv(
        f"{base_dir}/train/train.csv", header=False, index=False
    )
    pd.DataFrame(validation_data).to_csv(
        f"{base_dir}/validation/validation.csv", header=False, index=False
    )
    pd.DataFrame(test_data).to_csv(
        f"{base_dir}/test/test.csv", header=False, index=False
    )

In [ ]:
s3_dataprep_split_code_uri = sagemaker.s3.S3Uploader.upload("preprocess_split.py", s3uri_code)
%store s3_dataprep_split_code_uri
s3_dataprep_split_code_uri

Create file to get processors of Data Wrangler and Processing (splitter):

In [ ]:
%%writefile ../6-Pipelines/my_labs_solutions/dataprep_solution.py
# DataPrep
import sagemaker
from sagemaker.processing import Processor
from sagemaker.sklearn.processing import SKLearnProcessor

def get_data_wrangler_processor(
    processing_instance_type,
    processing_instance_count,
    role,
    base_job_prefix="CustomerChurnDataWrangler"
) -> Processor:
    
    sm_sess = sagemaker.session.Session()
    dw_container_uri = "415577184552.dkr.ecr.us-east-2.amazonaws.com/sagemaker-data-wrangler-container:1.x"
    
    # Processing step for feature engineering
    processor = Processor(
        role=role,
        image_uri=container_uri,
        instance_count=2,
        instance_type=instance_type,
        sagemaker_session=sess
    )
    return processor


def get_data_split_processor(
    processing_instance_type,
    processing_instance_count,
    role,
    base_job_prefix="CustomerChurn"
) -> SKLearnProcessor:
    
    sm_sess = sagemaker.session.Session()
    
    # Processing step for feature engineering
    sklearn_processor = SKLearnProcessor(
        framework_version="0.23-1",
        instance_type=processing_instance_type,
        instance_count=processing_instance_count,
        base_job_name=f"{base_job_prefix}/sklearn-CustomerChurn-preprocess",  # choose any name
        sagemaker_session=sm_sess,
        role=role,
    )
    return sklearn_processor

---
# [You can now go to the second lab 2-Modeling](../2-Modeling/modeling.ipynb)